In [3]:
from ete3 import Tree

In [17]:
t = Tree('test.nh', format=0)

In [11]:
s = []
for n in t.traverse():
    if not n.is_leaf():
        if all(child.is_leaf() and round(child.dist, 3) == 0 for child in n.children):
            s.append(n.children[0])
        else:
            for c in n.children:
                if c.is_leaf():
                    s.append(c)

In [4]:
names_cl = []
with open('tree_cl_not_phylip.fasta', 'r') as f:
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            names_cl.append(line.lstrip('>'))

In [6]:
s.extend(names_cl)

In [12]:
t.prune(s, preserve_branch_length=True)

In [7]:
len(s)

9497

In [13]:
len(t.get_leaves())

7227

In [19]:
with open('Colors/boot_test.nh', 'w') as f:
    f.write(t.write())

In [18]:
for i in t.iter_descendants():
    if not i.is_leaf() and i.support < 0.05:
        i.delete()

In [25]:
for i in t.iter_leaves():
    for j in i.get_sisters():
        if j.is_leaf() and i.get_distance(j) < 0.03:
            i.delete()
            break

Мы удалили слишком похожие листья, осталось примерно половина (6999) листьев с которыми мы и будем работать.

In [30]:
s = []
for i in t.iter_leaf_names():
    s.append(i)

In [34]:
with open('LOVs.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.strip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [35]:
new_fasta = {}
for i in s:
    new_fasta[i] = fasta[i]

In [40]:
with open('LOVs_chosen_for_small_tree.fasta', 'w') as f:
    new_file = []
    for k, v in new_fasta.items():
        new_file.append('>' + k + '\n' + ''.join(v) + '\n')
    f.write(''.join(new_file))

In [24]:
n1, n2 = t.search_nodes(name='Q9HPU80')[0], t.search_nodes(name='F7PMY20')[0]

In [33]:
n3 = t.search_nodes(name='A2W4QJ370')[0]

In [34]:
n1.get_distance(n3)

1.203944

###  Пробное дерево

In [174]:
t = Tree('(((H:0,K:0):0.5,(F:1,I:1):0):1,E:2);', format=5)

In [99]:
with open('LOV_tree_2', 'r') as f:
    line = f.readline()

In [105]:
l = line.split(':')
l = [i.split(',') for i in l]

In [81]:
print(t.write())

(((F:1,I:1)1:0,H:0.5)1:1,E:2);


In [156]:
t.up

In [179]:
for i in t.iter_leaves():
    print(i.name, i.get_sisters())
   

H [Tree node 'K' (0x7f34d336bc5)]
K [Tree node 'H' (-0x7ffff80cb2cc9438)]
F [Tree node 'I' (-0x7ffff80cb2cc9470)]
I [Tree node 'F' (0x7f34d336ba9)]
E [Tree node '' (-0x7ffff80cb2cc94a1)]


In [180]:
print(t)


         /-H
      /-|
     |   \-K
   /-|
  |  |   /-F
--|   \-|
  |      \-I
  |
   \-E


### Nexus to Newik

In [12]:
with open('Colors/nextonewik.txt', 'r') as f:
    s = f.readline()

In [13]:
s

'((((A126X2Z60[&!color=#00ff00]:0.030288,A126X4500[&!color=#00ff00,circlesize=10]:0.045336)[&boot=0.18]:0.013083,(((A126WVG20[&!color=#00ff00]:0.01361,(A126WYG20[&!color=#00ff00]:0.076647,A126X3280[&!color=#00ff00]:0.02391)[&boot=0.32]:0.023343)[&boot=0.1]:0.020303,(A126WWR10[&!color=#00ff00]:0.078921,(A1Q3BTV70[&!color=#00ff00]:0.056422,(F6GTW20[&!color=#00ff00]:0.055381,(A126WVJ10[&!color=#00ff00]:0.072078,A126X1U50[&!color=#00ff00]:0.033664)[&boot=0.4]:0.021007)[&boot=0.08]:0.010971)[&boot=0.16]:0.010484)[&boot=0.14]:0.011315)[&boot=0.02]:0.0,(A126WV260[&!color=#00ff00]:0.046414,A126X0540[&!color=#00ff00]:0.033439)[&boot=0.16]:0.010516)[&boot=0.0]:0.011037)[&boot=0.01]:0.011156,((((((A126X1Y50[&!color=#00ff00]:0.056311,(A067GCE70[&!color=#00ff00]:0.041631,(A126WVL60[&!color=#00ff00]:0.035036,(A126X0V20[&!color=#00ff00]:0.045413,A126WYP40[&!color=#00ff00]:0.045744)[&boot=0.43]:0.033247)[&boot=0.37]:0.026951)[&boot=0.11]:0.0)[&boot=0.16]:0.027195,((A067LAA20[&!color=#00ff00]:0.022688,

In [14]:
c = str()
i=0
while i < len(s):
    if s[i] == '[' and s[i+1:i+6] != '&boot':
        while s[i] != ']':
            i+= 1
        i+=1
    elif s[i] == '[' and s[i+1:i+6] == '&boot':
        while s[i] != '=':
            i+=1
        i+=1
        while s[i] != ']':
            c += s[i]
            i+= 1
        i+=1
    else:
        c += s[i]
        i+=1
        

In [15]:
with open ('test.nh', 'w') as f:
    f.write(c)